In [1]:
import numpy as np
# import tensorflow.keras as keras

from keras.optimizers import Adam
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import LeakyReLU, UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model

import matplotlib.pyplot as plt
import scipy
import datetime

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_dir='data/'
raw_data_dir=data_dir+'raw/'

def get_img(i,target_size):
    import imageio, skimage
    x = imageio.imread(raw_data_dir+'data_'+str(i)+'_x.jpg',as_gray=True).astype(np.float)
    x = scipy.misc.imresize(x, target_size)
    x= np.stack((x,)*1, -1)
    x = x/127.5 - 1.
    
    y = imageio.imread(raw_data_dir+'data_'+str(i)+'_y.jpg', as_gray=True).astype(np.float)
    y = scipy.misc.imresize(y, target_size)
    y= np.stack((y,)*1, -1)
    y = y/127.5 - 1.

    return x,y

def load_images(target_size):
    from os import listdir
    from os.path import isfile, join
    img_files = [f for f in listdir(raw_data_dir) if isfile(join(raw_data_dir, f)) and '_x.jpg' in f]
    n_x=len(img_files)
    X=np.zeros((n_x,target_size[0], target_size[1], target_size[2]))
    Y=np.zeros((n_x,target_size[0], target_size[1], target_size[2]))
    
    for i in range(n_x):
        x,y =get_img(i,target_size)
        X[i,:,:,:] = x
        Y[i,:,:,:] = y
    return X,Y,n_x

In [4]:
# SOURCE https://github.com/eriklindernoren/Keras-GAN

def build_generator(input_shape,gf,name):
    """U-Net Generator"""
    
    n_H, n_W, n_C = input_shape
    def conv2d(layer_input, filters, f_size=4):
        """Layers used during downsampling"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        d = InstanceNormalization()(d)
        return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
        """Layers used during upsampling"""
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = InstanceNormalization()(u)
        u = Concatenate()([u, skip_input])
        return u

    # Image input
    d0 = Input(shape=input_shape)

    # Downsampling
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

    # Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(n_C, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img, name=name)

def build_discriminator(input_shape, df, name):

    def d_layer(layer_input, filters, f_size=4, normalization=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            d = InstanceNormalization()(d)
        return d

    img = Input(shape=input_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity, name=name)

In [5]:
def build_model(input_shape, gf=32, df=64, name='cycleGAN_emoji'):
    n_H, n_W, n_C = input_shape
    # Calculate output shape of D (PatchGAN)
    patch = int(n_H / 2**4)
    disc_patch = (patch, patch, 1)
    
    # Build and compile the discriminators
    d_A = build_discriminator(input_shape, df, name="d_A")
    d_B = build_discriminator(input_shape, df, name='d_B')
    
    #-------------------------
    # Construct Computational
    #   Graph of Generator
    #-------------------------
    # Build the generators
    g_AB = build_generator(input_shape,gf,name='g_AB')
    g_BA = build_generator(input_shape,gf,name='g_BA')
    # Input images from both domains
    img_A = Input(shape=input_shape,name='input_img_A')
    img_B = Input(shape=input_shape,name='input_img_B')

    # Translate images to the other domain
    fake_B = g_AB(img_A)
    fake_A = g_BA(img_B)
    # Translate images back to original domain
    reconstr_A = g_BA(fake_B)
    reconstr_B = g_AB(fake_A)
    # Identity mapping of images
    img_A_id = g_BA(img_A)
    img_B_id = g_AB(img_B)

    # For the model model we will only train the generators
    #NOTE: y setting trainable=False after the discriminator has been compiled 
    #the discriminator is still trained during discriminator.train_on_batch but 
    #since it's set to non-trainable before the model model is compiled it's not 
    #trained during model.train_on_batch.
    #d_A.trainable = False
    #d_B.trainable = False

    # Discriminators determines validity of translated images
    valid_A = d_A(fake_A)
    valid_B = d_B(fake_B)

    # Combined model trains generators to fool discriminators
    model = Model(inputs=[img_A, img_B],
                          outputs=[ valid_A, valid_B,
                                    reconstr_A, reconstr_B,
                                    img_A_id, img_B_id ],name=name)

    return model


In [6]:
def train_epoch(model,imgs_A, imgs_B, epochs=1, batch_size=1):

    start_time = datetime.datetime.now()
    # Adversarial loss ground truths
    m, n_H, n_W, n_C = imgs_A.shape
    # Calculate output shape of D (PatchGAN)
    patch = int(n_H / 2**4)
    disc_patch = (patch, patch, 1)

    valid = np.ones((m,) + disc_patch)
    fake = np.zeros((m,) + disc_patch)
    # ----------------------
    #  Train Discriminators
    # ----------------------
    # Translate images to opposite domain
    fake_B = model.get_layer("g_AB").predict(imgs_A)
    fake_A = model.get_layer("g_BA").predict(imgs_B)
    # Train the discriminators (original images = real / translated = Fake)
    model.get_layer("g_AB").trainable=False
    model.get_layer("g_BA").trainable=False
    model.get_layer("d_A").trainable=True
    model.get_layer("d_B").trainable=True
    dA_loss_real = model.get_layer("d_A").fit(imgs_A, valid, batch_size=batch_size, epochs=epochs, verbose=0)
    dA_loss_fake = model.get_layer("d_A").fit(fake_A, fake, batch_size=batch_size, epochs=epochs, verbose=0)

    dB_loss_real = model.get_layer("d_B").fit(imgs_B, valid, batch_size=batch_size, epochs=epochs, verbose=0)
    dB_loss_fake = model.get_layer("d_B").fit(fake_B, fake, batch_size=batch_size, epochs=epochs, verbose=0)
    # ------------------
    #  Train Generators
    # ------------------
    # Train the generators
    model.get_layer("g_AB").trainable=True
    model.get_layer("g_BA").trainable=True
    model.get_layer("d_A").trainable=False
    model.get_layer("d_B").trainable=False
    g_loss = model.fit([imgs_A, imgs_B],
                                            [valid, valid,
                                            imgs_A, imgs_B,
                                            imgs_A, imgs_B],
                                  batch_size=batch_size,
                                  epochs=epochs)
    
    return model,g_loss,dA_loss_real,dA_loss_fake,dB_loss_real,dB_loss_fake

In [7]:
def sample_images(g_AB, g_BA, imgs_A, imgs_B,epoch):
    m, n_H, n_W, _ = imgs_A.shape
    figsize=(n_H,n_W)
    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)
    
    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(m, len(titles),figsize=figsize)
    for r in range(m):
        axs[r,0].imshow(imgs_A[r,:,:,0], cmap='gray')
        axs[r,0].set_title(titles[0])
        axs[r,0].axis('off')
        axs[r,1].imshow(fake_B[r,:,:,0], cmap='gray')
        axs[r,1].set_title(titles[1])
        axs[r,1].axis('off')
        axs[r,2].imshow(reconstr_A[r,:,:,0], cmap='gray')
        axs[r,2].set_title(titles[2])
        axs[r,2].axis('off')
    fig.savefig("output/cycleGAN_epoch_%d.png" % (epoch))
    plt.close()

In [8]:
input_shape=[256,256,1]
X,Y,n_x = load_images(input_shape)

In [9]:
def get_compiled_model(model_filepath=None):
    if model_filepath is None :
        model=build_model(input_shape,name='model_combined')
    else :
        model = load_model(model_filepath)
        
    optimizer = Adam(0.0002, 0.5)
    model.get_layer('d_A').compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
    model.get_layer('d_B').compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])

    model.compile(loss=['mse', 'mse','mae', 'mae','mae', 'mae'],
                loss_weights=[1, 1, 10.0, 10.0, 1.0, 1.0],
                optimizer=optimizer)
    return model

In [10]:
# init model
model_filepath='saved_model/cycleGAN_emoji_model__epoch_last.h5'
model=get_compiled_model()
model.save(filepath=model_filepath,overwrite=False)

[WARNING] saved_model/cycleGAN_emoji_model__epoch_last.h5 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [ ]:
np.random.seed(3)
m, _, _, _ = X.shape
_s=np.random.randint(m-5)
X_sample=X[_s:_s+5,:,:,:]
Y_sample=Y[_s:_s+5,:,:,:]
model = get_compiled_model(model_filepath)
for i in range(0, 150):
    model,_,_,_,_,_=train_epoch(model,X, Y, epochs=1, batch_size=16)
    model.save(filepath=model_filepath,overwrite=True)
    sample_images(model.get_layer('g_AB'),model.get_layer('g_BA'),X_sample, Y_sample,i)
    model = get_compiled_model(model_filepath)
